# DicoCo

*Mathieu Dussaillant*

In [1]:
import re
import pandas as pd
from IPython.display import clear_output, HTML, Markdown 
import ipywidgets as widgets
from IPython import display
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re


In [2]:
dfDicoBuch = pd.read_csv("./../data/dictionary/DFfin.csv", delimiter=";", encoding="utf8" )

In [3]:
# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dataframe/BigiRichFull2.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBigiFull = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

In [5]:
dfFin = dfDicoBuch.merge(dfDicoBigiFull,how='left', left_on='headword', right_on='hwd')

In [7]:
# import necessaire à la création des widgets


In [8]:
def filtDf(strHwd):    
    dfDicoBuchFiltered = dfFin[dfFin['headword'].str.contains(strHwd, na=False,flags=re.IGNORECASE, regex=True)]
    
    return dfDicoBuchFiltered

In [9]:
def getMp3(word):
    session = HTMLSession()
    response = session.get("https://www.ldoceonline.com/dictionary/"+word)
    soup = BeautifulSoup(response.text,'html.parser')
    souplettesBody = soup.find('body')
    
    finded3 = souplettesBody.findAll('span',{"class":re.compile('(frequent )*Head')})
    
    listmp3Br = []
    listmp3Am = []
    listPOS = []
    listPRON = []
    
    for v in finded3:
        if v.find('span', {'class':'POS'}) is not None:
            soupletteV = v.find('span',{"class":"POS"})
            vJPG = soupletteV.text.strip()
            listPOS.append(vJPG)

        if v.find('span', {'class':'PRON'}) is not None:
            pronV = v.find('span',{"class":"PRON"})
            vPRON = pronV.text.strip()
            listPRON.append(vPRON)
            
        spans = v.findAll('span')
        for span in spans:
            if span.has_attr('data-src-mp3'):
                if 'British' in span['title']:
                    listmp3Br.append(span['data-src-mp3'].split('?', 1)[0])
                if 'American' in span['title']:
                    listmp3Am.append(span['data-src-mp3'].split('?', 1)[0])
                    
        
        
    
    listMots = len(listPOS) * [word]
    
    while (len(listPRON) < len(listPOS)):
        listPRON.append(listPRON[0])

    dfResults = pd.DataFrame(list(zip(listMots, listPOS, listPRON, listmp3Br, listmp3Am)),
                    columns = ['WORD', 'POS', 'PRON', 'MP3br', 'MP3us'])

    return dfResults

In [10]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="love",
       description='filtre hwd',
       layout={'width': '300px'} )

In [11]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')
c_slider = widgets.IntSlider(min=0, max=20, step=1, value=2, description='Max occ:')
# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(param=''):
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = textTarget.value

    with out1:
        clear_output()
        
    with out2:
        clear_output()
        display.display(filtDf(targetWord).head(c_slider.value))
        
    with out3:
        clear_output()
        for word in filtDf(targetWord)['headword'].head(c_slider.value):
            dfWord = getMp3(word)
            
            for i in range(len(dfWord)):
                display.display(HTML(' Word : <b>'+dfWord.iloc[i]['WORD']+'</b>'))
                display.display(HTML(' POS : '+dfWord.iloc[i]['POS']))
                display.display(HTML(' PRON : '+dfWord.iloc[i]['PRON']))
                display.display(HTML('Prononciation <b>britanique</b>'))
                display.display(display.Audio(url=dfWord.iloc[i]['MP3br']))
                display.display(HTML('Prononciation <b>américaine</b>'))
                display.display(display.Audio(url=dfWord.iloc[i]['MP3us']))

        
        
        
button.on_click(on_button_clicked)

In [12]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget, button,c_slider])

# les widgets d'output
vbOuts = widgets.VBox([out1,out2, out3])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts])
hbAllWidgets